<a href="https://colab.research.google.com/github/Mohamed-Silaya/ML-ZAKA/blob/main/Copy_of_04_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© 2022, Zaka AI, Inc. All Rights Reserved

#Regression
**Objective:** In this notebook exercise, we will work in the Boston House Price dataset to predict through regression the price of houses in thousand of dollars. 
We will load the data, create a baseline model, train and evaluate it to predict with it and finally alter the performance of our model by standardizing our dataset and trying out wider and/or deeper network topologies.

### Importing the data from the github repository

In [1]:
# clone git repo
!git clone https://github.com/zaka-ai/intro2dl.git

# change directory
%cd intro2dl/data/

Cloning into 'intro2dl'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.
/content/intro2dl/data


## Regression with Boston House Price dataset

### 1. Load data

In this notebook, we are going to use the [**Boston house price dataset** dataset](https://archive.ics.uci.edu/ml/datasets/Housing). 

The dataset describes 13 numerical properties of houses in Boston suburbs and is concerned with modeling the price of houses in those suburbs in thousands of dollars. As such, this is a regression predictive modeling problem. Input attributes include things like crime rate, proportion of nonretail business acres, chemical concentrations and more.



In [2]:
from pandas import read_csv

# load dataset
dataframe = read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

### 2. Define Base Model

Create a Keras model with 1 hidden layer (size = input layer size).

We should define a `baseline_model()` funtion that will create the model, compile it and return it.

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# define baseline model
def baseline_model():
	model = Sequential()
	model.add(Dense(13, input_dim=13, activation="relu"))
	model.add(Dense(1, activation="linear"))
 
	model.compile(loss="mean_squared_error", optimizer="adam")

	return model

### 3. Evaluate baseline model

Evaluate the model using stratified cross validation in the scikit-learn framework. Number of splits should be 10. 

In [6]:
! pip install scikeras
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# evaluate model
estimator = KerasRegressor(model=baseline_model, epochs=10, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=42, shuffle = True)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Baseline: 0.07 (0.49) MSE


## Lift Performance By Standardizing The Dataset
Standardizing the dataset referes to transforming all datapoints values to the range of 0 to 1. This is done using `scikit-learn`'s StandardScaler. We will also build a pipeline which will call the function creating the model, then compile and training it. The last step is to evaluate the performance of the model using cross-validation. This will show us whether standardizing a dataset betters the performance of a deep learning model.

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=baseline_model, epochs=10, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standarized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standarized: 0.27 (0.19) MSE


## Tune The Neural Network Topology
We can alter the architecture of the hidden layers of the neural network to observe changes in the results we get.

### Evaluate a wider network
A wider network is a network where the hidden layer has more neurons than it previously had. Let's create a network which has 25 neurons in the hidden layer instead of thirteen. So, almost double.

In [8]:
def wider_model():
	model = Sequential()
	model.add(Dense(25, input_dim=13, activation="relu"))
	model.add(Dense(1, activation="linear"))
 
	model.compile(loss="mean_squared_error", optimizer="adam")

	return model

Next up, standardizing the dataset and using the pipeline to build, compile and train the model to get the score of this wider network.

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=wider_model, epochs=10, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std())) 



Wider: 0.53 (0.14) MSE


### Evaluate a deeper network
A deeper network is a network which has more hidden layers than the previous baseline network. Let's add another hidden layer for a total of two hidden layers with 13 neurons each and check the results we get.

In [10]:
def deeper_model():
	model = Sequential()
	model.add(Dense(13, input_dim=13, activation="relu"))
	model.add(Dense(13, activation="relu"))
	model.add(Dense(1, activation="linear"))
 
	model.compile(loss="mean_squared_error", optimizer="adam")

	return model

Standardizing the dataset and using the pipeline to build, compile and train the model to get the score of this wider network.

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=deeper_model, epochs=10, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Deeper: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Deeper: 0.68 (0.12) MSE
